In [1]:
import pandas as pd
import ClashFun as CF 

import plotly.express as px
import plotly.graph_objects as go
#from ipywidgets import widgets

path = "./data/"

clash_csv = path+"clash_data.csv"

# file_smile_z  ="/home/nhomberg/Documents/data_PHD/human/output_smile/seq10_7sub0.shuffle"
# file_smile_z  ="./data/seq10_6_8sub0.shuffle"
# file_smile_z2  ="./data/coincident6-8sub0.shuffle"

file_smile_z7  ="./data/seq10_7sub0.shuffle"
file_smile_z2_7  ="./data/coincident7sub0.shuffle"

file_name_short68 = 'seq10_6_8sub0'
file_name_long68='coincident6-8sub0'

file_name_short7 = 'seq10_7sub0'
file_name_long7='coincident7sub0'


# min_target= 10 
# motif_len_min = 6 
# motif_len_max = 8

thres_zscore_short_value =10 
thres_zsocre_long_value = 20 

# =============================================================================
# import de donnée 
# =============================================================================

df_clash = CF.read_clash_csv2df(clash_csv)

nb_mirna_total = len(set(df_clash.microRNA_name))


# df_smile_short_seq68 = CF.parce_read_smile(file_smile_z,df_clash,motif_len_min,motif_len_max,min_target)
# df_smile_fullseq68 = CF.parce_read_smile(file_smile_z2,df_clash,motif_len_min,motif_len_max,min_target)

# df_smile_short_seq7 = CF.parce_read_smile(file_smile_z7,df_clash,7,7,min_target)
# df_smile_fullseq7 = CF.parce_read_smile(file_smile_z2_7,df_clash,7,7,min_target)


# print(path+file_name_short68+'csv')
df_smile_short_seq68 = pd.read_csv(path+file_name_short68+'.csv')
df_smile_fullseq68 = pd.read_csv(path+file_name_long68+'.csv')

df_smile_short_seq7 = pd.read_csv(path+file_name_short7+'.csv')
df_smile_fullseq7 = pd.read_csv(path+file_name_long7+'.csv')



In [3]:

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import widgets


pd.options.plotting.backend = "plotly"


df_plot=CF.create_db_for_bar_plotly(df_smile_short_seq68,df_smile_fullseq68,
                                    "short_seq","long_seq",thres_zscore_short_value,thres_zsocre_long_value)


#print(df_plot)

#df = pd.DataFrame(dict(a=[1,3,2], b=[3,2,1]))
#df2 = pd.DataFrame(dict(a=[10,30,20], b=[30,20,10]))

 
fig1 = go.Bar(x= df_plot.index, y = df_plot['all'],name='all motif',
              customdata =  np.transpose([df_plot['nb_mirna']]),        
              texttemplate="%{y}  mirnas : %{customdata[0]}  ",
              hovertemplate="<br>".join([
            "total: %{y}","nombre de mirna trouvé :%{customdata[0]} "
        ]),textposition='auto')
fig2 = go.Bar(x= df_plot.index, y = df_plot['matching'],name ='matching motif',
              customdata = np.transpose([(df_plot['matching']/ df_plot['all'])*100 ]),
              texttemplate="%{y}   (%{customdata[0]:.2f}%)",
            textposition='auto')
#fig3 = go.Bar(x= df_plot.index, y = df_plot['matching'])



# slicer et case
thres_zscore_short = widgets.IntSlider(
    value=10.0,
    min=0.0,
    max=50.0,
    step=1.0,
    description='Z-score limite :',
    continuous_update=False
)
use_zscore_short = widgets.Checkbox(
    description='filter z-score for short_seq: ',
    value=True,
)

toggle68_7 = widgets.Checkbox(
    description='Togle motifs 6-8 / 7',
    value=True,
)

container = widgets.HBox(children=[use_zscore_short, thres_zscore_short])

thres_zscore_long = widgets.IntSlider(
    value=20.0,
    min=0.0,
    max=60.0,
    step=1.0,
    description='Z-score limite long :',
    continuous_update=False
)
use_zscore_long = widgets.Checkbox(
    description='filter z-score for long_seq: ',
    value=True,
)

container2 = widgets.HBox(children=[use_zscore_long, thres_zscore_long])




g = go.FigureWidget(
                    data=[fig1,fig2],
                    layout=go.Layout(
                        title=dict(
                            text=f'Motif of lenghts 6-8 found on short and long séquences, and the number of mirnas found out of {nb_mirna_total} mirnas total'
                        ),
                        barmode='overlay'
                    ))




# fonction d'update du graphique 
def response(change):
    
    if (use_zscore_long.value):
        z_score_long = thres_zscore_long.value
    else : 
        z_score_long = None
    if (use_zscore_short.value):
        z_score_short = thres_zscore_short.value 
    else : 
        z_score_short = None
    
    
    if(toggle68_7.value):
        g.update_layout(title=f"Motif of lenghts 6-8 found on short and long séquences, and the number of mirnas found out of {nb_mirna_total} mirnas total")
        df_smile_short_seq = df_smile_short_seq68
        df_smile_fullseq = df_smile_fullseq68
    else:
        g.update_layout(title=f"Motif of lenghts 7 found on short and long séquences, and the number of mirnas found out of {nb_mirna_total} mirnas total")
        df_smile_short_seq = df_smile_short_seq7
        df_smile_fullseq = df_smile_fullseq7
    


    df_plot=CF.create_db_for_bar_plotly(df_smile_short_seq,df_smile_fullseq,
                                        "short_seq","long_seq",z_score_short,z_score_long)
    with g.batch_update():

        g.data[0].x= df_plot.index
        g.data[0].y = df_plot['all']
        g.data[0].customdata =   np.transpose([df_plot['nb_mirna']])   
        g.data[0].texttemplate="%{y}  mirnas : %{customdata[0]}"

        g.data[1].x= df_plot.index
        g.data[1].y = df_plot['matching']
        g.data[1].customdata = np.transpose([(df_plot['matching']/ df_plot['all'])*100 ]) 
        g.data[1].texttemplate="%{y} (%{customdata[0]:.2f}%)"

# observer 
thres_zscore_short.observe(response, names="value")
use_zscore_short.observe(response, names="value")
thres_zscore_long.observe(response, names="value")
use_zscore_long.observe(response, names="value")
toggle68_7.observe(response, names="value")

# final verticla diplsay 
widgets.VBox([ toggle68_7, container,container2, g]) 

# test